In [1]:
# Comparative modeling by the automodel class
from modeller import *              # Load standard Modeller classes
from modeller.automodel import *    # Load the automodel class
import os
import sys

# Make modeller environment
env = Environ()
env.io.atom_files_directory = ['.']
# Read in HETATM from template PDB (ligand)
env.io.hetatm = True


                         MODELLER 10.4, 2022/10/28, r12463

     PROTEIN STRUCTURE MODELLING BY SATISFACTION OF SPATIAL RESTRAINTS


                     Copyright(c) 1989-2022 Andrej Sali
                            All Rights Reserved

                             Written by A. Sali
                               with help from
              B. Webb, M.S. Madhusudhan, M-Y. Shen, G.Q. Dong,
          M.A. Marti-Renom, N. Eswar, F. Alber, M. Topf, B. Oliva,
             A. Fiser, R. Sanchez, B. Yerkovich, A. Badretdinov,
                     F. Melo, J.P. Overington, E. Feyfant
                 University of California, San Francisco, USA
                    Rockefeller University, New York, USA
                      Harvard University, Cambridge, USA
                   Imperial Cancer Research Fund, London, UK
              Birkbeck College, University of London, London, UK


Kind, OS, HostName, Kernel, Processor: 4, Darwin w248h051.wireless.ugent.be 22.6.0 arm64
Date and time of com

In [6]:
import shutil
source_directory = '.'
destination_directory = 'second_run'
if not os.path.exists(destination_directory):
    os.makedirs(destination_directory)
files_before = os.listdir(source_directory)

In [7]:
# Define vars
pdb_code='6jp5_A'
target_uniprot_code='Q13698'
aln_filename=f'{pdb_code}-{target_uniprot_code}.ali'


In [8]:
# align target sequence to known, save file
aln = Alignment(env)
mdl = Model(env, file=f'./{pdb_code}.pdb')
aln.append_model(mdl=mdl, align_codes=pdb_code)
aln.append(file = f'./{target_uniprot_code}.fasta', align_codes=target_uniprot_code)
aln.salign()
aln.write(file=aln_filename, alignment_format='PIR')

openf___224_> Open           ./6jp5_A.pdb

Dynamically allocated memory at amaxcoordinates [B,KiB,MiB]:     50619344   49432.953    48.274

Dynamically allocated memory at    amaxsequence [B,KiB,MiB]:     50619420   49433.027    48.274

Dynamically allocated memory at amaxcoordinates [B,KiB,MiB]:     50619591   49433.195    48.275

Dynamically allocated memory at amaxcoordinates [B,KiB,MiB]:     50620441   49434.023    48.275

Dynamically allocated memory at amaxcoordinates [B,KiB,MiB]:     50621699   49435.254    48.277

Dynamically allocated memory at amaxcoordinates [B,KiB,MiB]:     50623603   49437.113    48.278

Dynamically allocated memory at    amaxsequence [B,KiB,MiB]:     50623643   49437.152    48.278

Dynamically allocated memory at amaxcoordinates [B,KiB,MiB]:     50623733   49437.238    48.279

Dynamically allocated memory at amaxcoordinates [B,KiB,MiB]:     50626589   49440.027    48.281

Dynamically allocated memory at    amaxsequence [B,KiB,MiB]:     50626649   49440.08

In [9]:
%%capture captured_output

# Do comparative modeling using AutoModel class
log.verbose()
a = AutoModel(env,
              alnfile=aln_filename,
              knowns=pdb_code,
              sequence=target_uniprot_code, 
              assess_methods=(assess.DOPE))
a.starting_model=1
a.ending_model=20
a.make()

%%writefile modeller-log.log
print(captured_output.stdout)

(if making multiple models)

In [10]:
# Get a list of all successfully built models from a.outputs
ok_models = [x for x in a.outputs if x['failure'] is None]

# Rank the models by DOPE score
key = 'DOPE score'
if sys.version_info[:2] == (2,3):
    ok_models.sort(key=lambda a,b: (a[key], b[key]))
else:
    ok_models.sort(key=lambda a: a[key])

# Get top model
m = ok_models[0]
print("Top model: %s (DOPE score %.3f)" % (m['name'], m[key]))

Top model: Q13698.B99990015.pdb (DOPE score -169556.469)


In [11]:
files_after = os.listdir(source_directory)
for file in files_after:
    if file not in files_before:
        source_file_path = os.path.join(source_directory, file)
        destination_file_path = os.path.join(destination_directory, file)
        # Move the file to the destination directory
        shutil.move(source_file_path, destination_file_path)